**Some tricks to combat overfitting**

* Part 1 [Intro to CNN (Dogs vs. Cats)](https://www.kaggle.com/imcr00z/intro-to-cnn-dogs-vs-cats)
* Part 2
* Part 3 [Transfer Learning (Dogs vs. Cats) 98% acc.](https://www.kaggle.com/imcr00z/transfer-learning-dogs-vs-cats-98-acc)

In this part i use augmentation and dropout layers to reduce overfitting and increase the predictive power of the model.

In [ ]:
import os
import tqdm
import shutil
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator

import logging
logger = tf.get_logger()
logger.setLevel(logging.ERROR)

In [ ]:
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Load data
Copied from my [Intro to CNN (Dogs vs. Cats)](http://https://www.kaggle.com/imcr00z/intro-to-cnn-dogs-vs-cats)

In [ ]:
# !rm -rf /kaggle/content

In [ ]:
CONTENT_DIR = '/kaggle/content'

TRAIN_DIR = CONTENT_DIR + '/train'
TRAIN_DIR_DOG = TRAIN_DIR + '/dog'
TRAIN_DIR_CAT = TRAIN_DIR + '/cat'

VALID_DIR = CONTENT_DIR + '/valid'
VALID_DIR_DOG = VALID_DIR + '/dog'
VALID_DIR_CAT = VALID_DIR + '/cat'

if not os.path.exists(CONTENT_DIR):
    import zipfile
    with zipfile.ZipFile('/kaggle/input/dogs-vs-cats/train.zip', 'r') as zipf:
        zipf.extractall(CONTENT_DIR)

    # Split cats and dogs images to train and valid datasets
    img_filenames = os.listdir(TRAIN_DIR)
    print('Num of images:', len(img_filenames))

    dog_filenames = [fn for fn in img_filenames if fn.startswith('dog')]
    cat_filenames = [fn for fn in img_filenames if fn.startswith('cat')]

    dataset_filenames = train_test_split(
        dog_filenames, cat_filenames, test_size=0.1, shuffle=True, random_state=42
    )

    train_dog_total, valid_dog_total, train_cat_total, valid_cat_total = [len(fns) for fns in dataset_filenames]
    train_total = train_dog_total + train_cat_total
    valid_total = valid_dog_total + valid_cat_total
    print('Train: {}, test: {}'.format(train_total, valid_total))

    # Move images
    make_dirs = [TRAIN_DIR_DOG, VALID_DIR_DOG, TRAIN_DIR_CAT, VALID_DIR_CAT]
    for dir, fns in zip(make_dirs, dataset_filenames):
        os.makedirs(dir, exist_ok=True)
        for fn in tqdm.tqdm(fns):
            shutil.move(os.path.join(TRAIN_DIR, fn), dir)
        print('elements in {}: {}'.format(dir, len(os.listdir(dir))))

# Augmentation
We change images to increase the size of the dataset.

In [ ]:
BATCH_SIZE = 128
IMAGE_SHAPE = 128

In [ ]:
def plot_images(images):
    fig, axes = plt.subplots(1, 5, figsize=(20, 20))
    for img, ax in zip(images, axes.flatten()):
        ax.imshow(img)
    plt.tight_layout()
    plt.show()    

In [ ]:
def show_augmentation(generator):
    image_data = image_generator.flow_from_directory(
        directory=TRAIN_DIR,
        target_size=(IMAGE_SHAPE, IMAGE_SHAPE),
        batch_size=BATCH_SIZE,
        shuffle=True
    )
    augmented_images = [image_data[0][0][0] for _ in range(5)]
    plot_images(augmented_images)

## Horisontal flipping

In [ ]:
image_generator = ImageDataGenerator(rescale=1./255, horizontal_flip=True)
show_augmentation(image_generator)

## Rotation

In [ ]:
image_generator = ImageDataGenerator(rescale=1./255, rotation_range=45)
show_augmentation(image_generator)

## Zoom

In [ ]:
image_generator = ImageDataGenerator(rescale=1./255, zoom_range=0.5)
show_augmentation(image_generator)

## Putting it all together

In [ ]:
image_generator = ImageDataGenerator(
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    rescale=1./255
)
train_data = image_generator.flow_from_directory(
    directory=TRAIN_DIR,
    target_size=(IMAGE_SHAPE, IMAGE_SHAPE),
    batch_size=BATCH_SIZE,
    shuffle=True,
    class_mode='binary'
)
show_augmentation(image_generator)

# Valid data

In [ ]:
valid_generator = ImageDataGenerator(rescale=1./255)
valid_data = valid_generator.flow_from_directory(
    directory=VALID_DIR,
    target_size=(IMAGE_SHAPE, IMAGE_SHAPE),
    batch_size=BATCH_SIZE,
    class_mode='binary',
    shuffle=False
)

# Model
With dropout layer.

In [ ]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(
        filters=32,
        kernel_size=(3, 3),
        activation='relu',
        input_shape=(IMAGE_SHAPE, IMAGE_SHAPE, 3)
    ),
    tf.keras.layers.MaxPooling2D(pool_size=2, strides=2),

    tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2, 2),

    tf.keras.layers.Conv2D(128, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2, 2),

    tf.keras.layers.Conv2D(128, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2, 2),
    
    # dropout layer
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(units=512, activation='relu'),
    tf.keras.layers.Dense(units=2, activation='softmax')
])

In [ ]:
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [ ]:
model.summary()

In [ ]:
EPOCHS = 60
history = model.fit_generator(
    generator=train_data,
    steps_per_epoch=(train_total + BATCH_SIZE - 1) // BATCH_SIZE,
    epochs=EPOCHS,
    validation_data=valid_data,
    validation_steps=(valid_total + BATCH_SIZE - 1) // BATCH_SIZE,
)

# Train visualization
Look at it and compare with the graph [here](http://https://www.kaggle.com/imcr00z/intro-to-cnn-dogs-vs-cats)

In [ ]:
plt.figure(figsize=(12, 8))

plt.subplot(1, 2, 1)
plt.plot(range(EPOCHS), history.history['accuracy'], label='train')
plt.plot(range(EPOCHS), history.history['val_accuracy'], label='valid')
plt.legend(loc='lower right')
plt.title('Accuracy')

plt.subplot(1, 2, 2)
plt.plot(range(EPOCHS), history.history['loss'], label='train')
plt.plot(range(EPOCHS), history.history['val_loss'], label='valid')
plt.legend(loc='upper right')
plt.title('Loss (sparse_categorical_crossentropy)')

plt.show()